In [313]:
import os

import datetime
import time

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import poisson

import matplotlib.pyplot as plt

In [314]:
route='LUX-CPH'
risk_spill=0.8
risk_spoil=risk_spill
unique_identifier=['DepDate','FltNum','dtime']
cluster_variables=['dday','dtime','Direction','month']

# Determine downweight factor

In [315]:
C_datadriven=pd.read_csv('~/Data/Intermediate_Output/R_Training_Pax.csv')
#C_datadriven=pd.read_csv('R_Training_Pax.csv')
C_datadriven=C_datadriven.groupby(cluster_variables)['NumPax'].sum().reset_index()
C_datadriven=C_datadriven.groupby(cluster_variables)['NumPax'].max().reset_index().rename(columns={'NumPax': 'downweight'})
C_datadriven['downweight']=C_datadriven['downweight'].astype('float')/0.8
C_datadriven['downweight']=C_datadriven['downweight'].apply(lambda x: 0.8 if x>0.8 else x)
C_datadriven['downweight']=0.8

In [316]:
# C_datadriven.loc[C_datadriven['month']==5,:]

# Import to total capacity

In [317]:
TotalCap=pd.read_csv('~/Data/FrontEnd_Input/Capacity_forPlots_'+route+'.csv')

In [318]:
Prediction=pd.read_csv('~/Data/Intermediate_Output/R_Output_Test_Pax.csv',sep=',')

# Associate downweight factor and cabin capacity to prediction

In [319]:
Prediction=Prediction.merge(C_datadriven,on=cluster_variables,how='left')
Prediction['downweight']=Prediction['downweight'].apply(lambda x: 0.8 if pd.isnull(x) else x)

In [320]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Prediction['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Prediction['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Prediction=Prediction.merge(Map_DepTime,on='dtime')
Prediction['dtime']=Prediction['deptime']
Prediction=Prediction[[x for x in Prediction.columns if x!='deptime']]

Prediction=Prediction.merge(TotalCap[unique_identifier+['Dprio','Cabin Capacity']],
on=unique_identifier+['Dprio'])

In [321]:
Prediction.shape

(330808, 13)

In [299]:
IdealCurve=Prediction[unique_identifier+['Dprio','forecast_bookings','Cabin Capacity','downweight']].copy()

In [300]:
IdealCurve=IdealCurve.merge(IdealCurve.groupby(unique_identifier)['forecast_bookings'].sum().reset_index()\
.rename(columns={'forecast_bookings': 'forecast_bookings_sum'}),on=['DepDate','FltNum','dtime'])

In [301]:
IdealCurve['forecast_bookings']=IdealCurve['forecast_bookings']/IdealCurve['forecast_bookings_sum']

In [302]:
IdealCurve=IdealCurve[[x for x in IdealCurve.columns if x!='forecast_bookings_sum']]

In [303]:
IdealCurve['lambda_100']=IdealCurve['Cabin Capacity']*IdealCurve['forecast_bookings']
IdealCurve['lambda_80']=IdealCurve['Cabin Capacity']*IdealCurve['downweight']*IdealCurve['forecast_bookings']

In [304]:
IdealCurve=IdealCurve[unique_identifier+['Dprio']+[x for x in IdealCurve.columns if 'lambda' in x]]

In [305]:
IdealCurve['Dprio']=-IdealCurve['Dprio']
IdealCurve=IdealCurve.set_index(['Dprio']+unique_identifier)

IdealCurve_cumul_100=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_100'].cumsum().reset_index().rename(columns={'lambda_100': 'lambda_100_cumsum'})
IdealCurve_cumul_100['Dprio']=-IdealCurve_cumul_100['Dprio']

IdealCurve_cumul_80=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_80'].cumsum().reset_index().rename(columns={'lambda_80': 'lambda_80_cumsum'})
IdealCurve_cumul_80['Dprio']=-IdealCurve_cumul_80['Dprio']

IdealCurve=IdealCurve.reset_index()
IdealCurve['Dprio']=-IdealCurve['Dprio']

IdealCurve=IdealCurve.set_index(['Dprio']+unique_identifier)

IdealCurve_cond_100=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_100'].cumsum().reset_index().rename(columns={'lambda_100': 'lambda_100_cond'})

IdealCurve_cond_80=IdealCurve.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['lambda_80'].cumsum().reset_index().rename(columns={'lambda_80': 'lambda_80_cond'})

IdealCurve=IdealCurve.reset_index()

IdealCurve_cumul_100=IdealCurve_cumul_100.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cumul_80=IdealCurve_cumul_80.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cond_100=IdealCurve_cond_100.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cond_100['thres_100']=poisson.ppf(1-risk_spoil,IdealCurve_cond_100['lambda_100_cond'])
IdealCurve_cond_80=IdealCurve_cond_80.sort_values(by=['Dprio']+unique_identifier)
IdealCurve_cond_80['thres_80']=poisson.ppf(risk_spoil,IdealCurve_cond_80['lambda_80_cond'])

In [306]:
IdealCurve_cond_100=IdealCurve_cond_100\
.merge(Prediction[['Dprio']+unique_identifier+['Cabin Capacity']],on=['Dprio']+unique_identifier)

IdealCurve_cond_80=IdealCurve_cond_80\
.merge(Prediction[['Dprio']+unique_identifier+['Cabin Capacity','downweight']],on=['Dprio']+unique_identifier)

In [307]:
IdealCurve_cond_100['Ideal_upper']=IdealCurve_cond_100['Cabin Capacity']-IdealCurve_cond_100['thres_100']
IdealCurve_cond_100['Ideal_upper']=IdealCurve_cond_100[['Ideal_upper','Cabin Capacity']].apply(lambda x: x[1] if x[0]>x[1] else x[0],axis=1)
IdealCurve_cond_80['Ideal_lower']=IdealCurve_cond_80['Cabin Capacity']*IdealCurve_cond_80['downweight']-IdealCurve_cond_80['thres_80']
IdealCurve_cond_80['Ideal_lower']=IdealCurve_cond_80['Ideal_lower'].apply(lambda x: 0 if x<0 else x)

In [308]:
IdealCurve=IdealCurve_cumul_100.merge(IdealCurve_cumul_80,on=['Dprio']+unique_identifier)\
.merge(IdealCurve_cond_80[['Dprio']+unique_identifier+['Ideal_lower']],on=['Dprio']+unique_identifier)\
.merge(IdealCurve_cond_100[['Dprio']+unique_identifier+['Ideal_upper']],on=['Dprio']+unique_identifier)

In [309]:
IdealCurve=IdealCurve[['Dprio']+unique_identifier+['Ideal_lower','lambda_80_cumsum','lambda_100_cumsum','Ideal_upper']]

In [310]:
IdealCurve.columns=['Dprio']+unique_identifier+['Ramp-up frontier','Ideal curve (80% LF)','Ideal curve (100% LF)','Phase-down frontier']

In [311]:
IdealCurve.to_csv('~/Data/Intermediate_Output/IdealCurve.csv',index=False)

# Risk and Actual Bookings

In [254]:
Risk=Prediction[['Dprio']+unique_identifier+['NumPax']].copy()

In [255]:
Mapping_StrToDt=pd.concat([Series(Risk['DepDate'].unique()),
Series([pd.to_datetime(x) for x in Risk['DepDate'].unique()])],axis=1)
Mapping_StrToDt.columns=['DepDate','DepDate_new']
Risk=Risk.merge(Mapping_StrToDt,on='DepDate')
Risk['DepDate']=Risk['DepDate_new']
Risk=Risk[[x for x in Risk.columns if x!='DepDate_new']]

In [256]:
Risk['Dprio']=-Risk['Dprio']
Risk=Risk.set_index(['Dprio']+unique_identifier)

Risk=Risk.groupby(level=[x for x in range(len(unique_identifier)+1)]).sum()\
.groupby(level=[x for x in range(1,len(unique_identifier)+1)])['NumPax'].cumsum().reset_index()\
.rename(columns={'NumPax': 'NumPax_cumsum'})

Risk['Dprio']=-Risk['Dprio']

In [257]:
Risk['IssueDate']=Risk['DepDate']-Risk['Dprio'].apply(lambda x: datetime.timedelta(x-1))

In [258]:
Actuals=Risk.loc[Risk['IssueDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

In [259]:
Actuals=Actuals[[x for x in Actuals.columns if x!='IssueDate']]
Actuals=Actuals.rename(columns={'NumPax_cumsum': 'Actual Bookings'})

In [260]:
Risk=Risk.loc[Risk['IssueDate']==pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

In [261]:
Risk=Risk[[x for x in Risk.columns if x!='IssueDate']]

In [262]:
Risk['DepDate']=Risk['DepDate'].apply(lambda x: str(x).split(' ')[0])

In [263]:
Risk=Risk.merge(Prediction[['Dprio']+unique_identifier+['Cabin Capacity','downweight']],
               on=['Dprio']+unique_identifier)

In [264]:
Risk=Risk.merge(IdealCurve_cond_100[['Dprio']+unique_identifier+['lambda_100_cond']],on=['Dprio']+unique_identifier)\
.merge(IdealCurve_cond_80[['Dprio']+unique_identifier+['lambda_80_cond']],on=['Dprio']+unique_identifier)

In [265]:
Risk['SpillageRisk']=1-poisson.cdf(Risk['Cabin Capacity']-Risk['NumPax_cumsum'],Risk['lambda_100_cond'])
Risk['SpoilageRisk']=poisson.cdf(Risk['Cabin Capacity']*Risk['downweight']-Risk['NumPax_cumsum'],Risk['lambda_80_cond'])

In [266]:
Risk=Risk.merge(IdealCurve[['Dprio']+unique_identifier+['Ramp-up frontier','Phase-down frontier']],
          on=['Dprio']+unique_identifier)

In [267]:
Risk['Intensity_downweighted']=Risk['NumPax_cumsum']-Risk['Ramp-up frontier']
Risk['Intensity_full']=Risk['NumPax_cumsum']-Risk['Phase-down frontier']

In [268]:
Risk=Risk[['DepDate','FltNum','dtime','SpoilageRisk','Intensity_downweighted','Intensity_full','SpillageRisk']]

# Data Export

In [269]:
Risk=Risk.merge(Prediction[unique_identifier+['Direction','month','dday']].drop_duplicates(subset=unique_identifier),on=unique_identifier)
Risk['Route']=route

MonthMapping=DataFrame([[x for x in range(1,13)],['January','February','March','April','May','June','July','August',
                'September','October','November','December']]).transpose()
MonthMapping.columns=['month','Month']
MonthMapping['month']=MonthMapping['month'].astype('int') 
Risk=Risk.merge(MonthMapping,on='month')
Risk['month']=Risk['Month']
Risk=Risk[[x for x in Risk.columns if x!='Month']]

Risk=Risk[['Route','DepDate', 'FltNum', 'dtime', 'Direction', 'month', 'dday',
       'SpoilageRisk', 'Intensity_downweighted', 'Intensity_full',
       'SpillageRisk']]

Risk=Risk.sort_values(by=['DepDate', 'FltNum', 'dtime', 'Direction'])

Risk.to_csv('~/Data/FrontEnd_Input/Risk_'+route+'.csv',index=False)

In [270]:
Actuals['DepDate']=Actuals['DepDate'].apply(lambda x: str(x).split(' ')[0])

Actuals=Actuals.merge(Prediction[unique_identifier+['Direction','month','dday']].drop_duplicates(subset=unique_identifier),
on=unique_identifier)
Actuals['Route']=route

Actuals=Actuals[['Route','DepDate', 'FltNum', 'dtime', 'Direction', 'month', 'dday', 'Dprio','Actual Bookings']]

Actuals['Actual Bookings']=Actuals['Actual Bookings'].apply(lambda x: x if x>=0 else 0)

Actuals.to_csv('~/Data/FrontEnd_Input/Actuals_'+route+'.csv',index=False)

In [271]:
IdealCurve=IdealCurve.merge(Prediction[unique_identifier+['Direction','month','dday']].drop_duplicates(subset=unique_identifier),
on=unique_identifier)

IdealCurve['Route']=route

IdealCurve=IdealCurve[['Route','DepDate', 'FltNum', 'dtime', 'Direction', 'month', 'dday', 'Dprio',
'Ramp-up frontier','Ideal curve (80% LF)','Ideal curve (100% LF)','Phase-down frontier']]

IdealCurve.to_csv('~/Data/FrontEnd_Input/IdealCurve_'+route+'.csv',index=False)